In [2]:
import influxdb
import pandas as pd
from influxdb import InfluxDBClient
import json
from datetime import datetime

In [3]:
telegraf_auth = json.load(open('telegraf_credentials'))
telegraf_client = InfluxDBClient(
    host="qtinfluxdb01.intra.qt.io",
    username=telegraf_auth['username'],
    password=telegraf_auth['password'],
    port=8086,
    ssl=True,
    database="telegraf")

In [ ]:
<h2>Open nebula waiting times raw</h2>
This notebooks retireves data about waiting times from opennebula, cleans it (removes nan and values <=0), 
resamples is by 10 minutes and creates 2 files:  
mean values (grouped by 10 min)  
max values (grouped by 10 min)

In [4]:
q = 'SELECT * from "onedb" WHERE  time > now() - 7d'
rs = telegraf_client.query(q)
tdf = pd.DataFrame(rs.get_points())


/home/anwojcie/.local/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [5]:
tdf.shape

(106249, 14)

In [6]:
tdf.head()

,time,ctime,eetime,estime,etime,id,instance,petime,pstime,retime,rstime,stime,uname,wtime
0,2022-01-19T15:13:08Z,1642605188,1.642609e+09,1.642609e+09,1642608945,173518,one-master-202105,1.642609e+09,1.642609e+09,1642608942,1642608874,1642608869,coin,3681.0
1,2022-01-19T15:13:08Z,1642605188,1.642609e+09,1.642609e+09,1642608947,173519,one-master-202105,1.642609e+09,1.642609e+09,1642608943,1642608874,1642608869,coin,3681.0
2,2022-01-19T15:13:08Z,1642605188,1.642609e+09,1.642609e+09,1642608970,173520,one-master-202105,1.642609e+09,1.642609e+09,1642608966,1642608876,1642608869,coin,3681.0
3,2022-01-19T15:13:09Z,1642605189,1.642609e+09,1.642609e+09,1642609011,173521,one-master-202105,1.642609e+09,1.642609e+09,1642609006,1642608874,1642608869,coin,3680.0
4,2022-01-19T15:13:09Z,1642605189,1.642609e+09,1.642609e+09,1642608963,173525,one-master-202105,1.642609e+09,1.642609e+09,1642608959,1642608877,1642608870,coin,3681.0


In [7]:
df = tdf.loc[(tdf['instance'] == "one-master-202105")]

In [8]:
df.shape

(106249, 14)

In [9]:
df = df[['time','wtime']]
df.shape

(106249, 2)

In [10]:
df.isnull().values.any()

True

In [11]:
df = df.dropna()
df.shape

(106116, 2)

In [15]:
df = df.loc[(df['wtime'] > 0)]
df.shape

(106110, 2)

In [16]:
df.to_csv('wtimes_7days.csv', index=False)

In [12]:
test = df.loc[(df['wtime'] < 0)]

In [14]:
test.shape

(6, 2)